# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [16]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
import numpy as np
from pathlib import Path
from dotenv import load_dotenv

In [17]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [18]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [19]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path, index_col="neighbourhood")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [20]:
# Getting the data from the top 10 expensive neighbourhoods
to_data_mean = to_data.reset_index().copy().drop(columns=["year"]).set_index("neighbourhood").groupby("neighbourhood").mean().sort_values("average_house_value", ascending=False)
top_ten = to_data_mean.iloc[:10]
 
# Calculate the mean number of dwelling types units per year
to_data_gb = to_data.groupby("year").sum().drop(columns=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"])

# Calculate the average monthly shelter costs for owned and rented dwellings
to_data_shelter_costs = (to_data[["shelter_costs_owned", "shelter_costs_rented"]]).groupby("year").mean().sort_values("year")

### Panel Visualization Functions

In [21]:
# Define Panel visualization functions
to_data_mean_neighb = to_data_mean.sort_values("neighbourhood")
df_locations_mean = pd.concat([df_neighbourhood_locations, to_data_mean_neighb], axis="columns", join="inner").reset_index()
def neighbourhood_map():
    map_info = px.scatter_mapbox(
        df_locations_mean,
        lat="lat",
        lon="lon",
        hover_data=df_locations_mean,
        hover_name="neighbourhood",
        size="average_house_value",
        color="average_house_value",
        color_continuous_scale="icefire",
        title="Average House Value in Toronto",
        zoom=9,
        width=850, 
        height=450
)
    return map_info

def create_bar_chart(data, title, xlabel, ylabel, color):
    to_data_type = data.hvplot.bar(
    title=(title),
    xlabel=(xlabel), 
    ylabel=(ylabel),
    color=(color),
    rot=90,
    width=600, height=500).opts(
    yformatter="%.0f")
    return to_data_type

def create_line_chart(data, title, xlabel, ylabel, color):
    to_data_shelter = data.hvplot.line(
    title=(title),
    xlabel=(xlabel), 
    ylabel=(ylabel),
    color=(color),
    width=550, 
    height=350
    )
    return to_data_shelter

average_house_value = (to_data[["average_house_value"]]).groupby("year").mean().sort_values("year")
def average_house_value_plot():
    average_house_value_fig = average_house_value.hvplot.line(
        x="year",
        y="average_house_value",
        xlabel="Year", 
        ylabel="Avg House Value",
        title="Average House Value in Toronto",
        width=480, 
        height=350,
    ).opts(line_color='blue', yformatter="%.0f")
    return average_house_value_fig

avg_per_neighb = (to_data[["neighbourhood", "average_house_value"]]).sort_values(["year","neighbourhood"]).reset_index()
list_of_neighbourhood = avg_per_neighb["neighbourhood"].values.tolist()
def choose_neighbourhood(neighbourhood):
    return neighbourhood

def interact_neighbourhood():
    interact(choose_neighbourhood, neighbourhood=list_of_neighbourhood)
    return interact

def average_value_by_neighbourhood():
    avg_per_neighb_plot = avg_per_neighb.hvplot.line(
        x="year",
        y="average_house_value",
        xlabel="Year", 
        ylabel="Avg House Value",
        title="Neighbourhood:",
        groupby='neighbourhood',
        width=600, 
        height=450).opts(line_color='blue', yformatter="%.0f")
    return avg_per_neighb_plot
    
to_data_all = to_data.reset_index()
to_data_all = to_data_all[["year", "neighbourhood", "single_detached_house", "apartment_five_storeys_plus", "movable_dwelling", "semi_detached_house", "row_house", "duplex", "apartment_five_storeys_less", "other_house"]].set_index("year")
def number_dwelling_types():
    dwelling_types = to_data_all.hvplot.bar(
        xlabel="Year", 
        ylabel="Dwelling Types Units",
        title="Neighbourhood:",
        groupby='neighbourhood',
        rot=90,
        width=600, 
        height=450).opts(line_color='blue', yformatter="%.0f")
    return dwelling_types

to_data_year = to_data.reset_index().copy()
def average_house_value_snapshot():
    fig = px.bar(
        to_data_year, 
        x="neighbourhood", 
        y="average_house_value", 
        color="average_house_value", 
        facet_row="year",
        facet_row_spacing=0.04,
        facet_col_spacing=0.04,
        height=1000, width=900,
        title="Average House Values in Toronto per Neighbourhood"
        )
    return fig

def top_most_expensive_neighbourhoods():
    to_data_exp_top10 = top_ten.hvplot.bar(
    x='neighbourhood', 
    y='average_house_value',
    xlabel="Neighbourhood", 
    ylabel="Avg House Value",
    title="Top 10 Expensive Neighbourhood in Toronto",
    rot=90,  
    width=600, 
    height=500).opts(
    yformatter="%.0f"
    )
    return to_data_exp_top10

to_data_expensive = to_data.loc[to_data["average_house_value"] >= 500000, :].reset_index()
def sunburts_cost_analysis():
    fig_1 = px.sunburst(to_data_expensive, path=['year', "neighbourhood"], values='average_house_value',
                  color='shelter_costs_owned', hover_data=['average_house_value'],
                  color_continuous_scale='Blues', title="Costs Analysis of Most Expensive Neighbourhood in Toronto per Year",
                  color_continuous_midpoint=np.average(to_data_expensive['shelter_costs_owned'], 
                                                       weights=to_data_expensive['shelter_costs_rented']))
    return fig_1

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [24]:
# Bar chart for 2001
create_bar_chart_01 = create_bar_chart(
    data = to_data_gb.iloc[0],
    title="Dwelling Types Units in Toronto in 2001",
    xlabel="2001", 
    ylabel="Dwelling Types Units",
    color="Red"
)

# Bar chart for 2006
create_bar_chart_06 = create_bar_chart(
    data = to_data_gb.iloc[1],
    title="Dwelling Types Units in Toronto in 2006",
    xlabel="2006", 
    ylabel="Dwelling Types Units",
    color="Blue"
)

# Bar chart for 2011
create_bar_chart_11 = create_bar_chart(
    data = to_data_gb.iloc[2],
    title="Dwelling Types Units in Toronto in 2011",
    xlabel="2011", 
    ylabel="Dwelling Types Units",
    color="Yellow"
)


# Bar chart for 2016
create_bar_chart_16 = create_bar_chart(
    data = to_data_gb.iloc[3],
    title="Dwelling Types Units in Toronto in 2016",
    xlabel="2016", 
    ylabel="Dwelling Types Units",
    color="magenta"
)

create_line_shelter_owned = create_line_chart(
    data = to_data_shelter_costs.iloc[:,0],
    title="Average Monthly Shelter Cost for Owned Dwellings in Toronto",
    xlabel="Year", 
    ylabel="Avg Monthly Shelter Costs",
    color="Blue"
)

create_line_shelter_rented = create_line_chart(
    data = to_data_shelter_costs.iloc[:,1],
    title="Average Monthly Shelter Cost for Rented Dwellings in Toronto",
    xlabel="Year", 
    ylabel="Avg Monthly Shelter Costs",
    color="Yellow",
)

average_house_value_plot = create_line_chart(
    data = average_house_value,
    title="Average House Value in Toronto",
    xlabel="Year", 
    ylabel="Avg House Value",
    color="Blue"
)

# Create a Title for the Dashboard
dashboard_title = "# Toronto Real Estate Analysis 2001 to 2016"

# Define a welcome text
welcome_column = pn.Column(
    "Here is a visual analysis of historical home values, dwelling types per neighbourhood and home costs in Toronto, Ontario, as per census data from 2001 to 2016. Explore specific data by clicking the other tabs above to futher explore the progression of the real estate market.",
    neighbourhood_map)

# Create the main dashboard
row_of_dwelling_types = pn.Row(create_bar_chart_01 + create_bar_chart_06) 
row_of_dwelling_types_2 = pn.Row(create_bar_chart_11 + create_bar_chart_16)
mkt_column = pn.Column(row_of_dwelling_types, row_of_dwelling_types_2)
costs_column = pn.Column(create_line_shelter_owned, create_line_shelter_rented, average_house_value_plot)

column_of_analysis_1 = pn.Column(average_value_by_neighbourhood, number_dwelling_types)
column_of_analysis_2 = pn.Column(average_house_value_snapshot)

row_of_analysis = pn.Row(column_of_analysis_1, column_of_analysis_2)
row_of_top_exp = pn.Row(top_most_expensive_neighbourhoods, sunburts_cost_analysis)

# Create a tab layout for the dashboard
dashboard = pn.Column(dashboard_title,
                      pn.Tabs(
                          ("Welcome", welcome_column),
                          ("Yearly Market Analysis", mkt_column),
                          ("Shelter Costs Vs. House Value", costs_column),
                          ("Neighbourhood Analysis", row_of_analysis),
                          ("Top Expensive Neighbourhood", row_of_top_exp)
))

## Serve the Panel Dashboard

In [25]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive07522')
        [1] Column
            [0] Row
                [0] HoloViews(Layout)
            [1] Row
                [0] HoloViews(Layout)
        [2] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [3] Row
            [0] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Row(name='interactive07562')
                            [0] HoloViews(DynamicMap, name='interactive07562')
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                                [1] VSpacer()
                [1] Column
                    [0] Column()
                    [1] Row
                        [0] Row(name='interactive07605')
                            [0] HoloViews(DynamicMap, name='interactive07605')
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                                [1] VSpacer()
            [1] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Plotly(Figure, name='interactive07655')
        [4] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive07663')
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive07799')